In [1]:
import pickle
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from functools import reduce

In [2]:
def queries_list(file_name='queries_all.txt'):
    
    queries = []
#     correct = dict()
    correct = []
    f = open(file_name)
    num = 0
    
    for query in f:
        
        query = query.lower()
        separated = query.split(sep='\t')
        
        if len(separated) == 2:
#             correct[separated[1][:-1]] = num
            correct.append((separated[0], separated[1][:-1]))
            queries.append(separated[1])
            
        else:
            queries.append(separated[0][:-1])
            
        num +=1
    
    return queries, correct

In [3]:
def flatten(x):
    result = []
#     print('x = ', x)
    for elem in x:
        if hasattr(elem, "__iter__") and not isinstance(elem, str) and not isinstance(elem, tuple) and len(elem)>1:
            result.extend(flatten(elem))
        else:
            result.append(elem)
    return result

In [4]:
def list_concat(list1, list2):
    
    len1 = len(list1)
    len2 = len(list2)
    
    return [[list1[i], list2[j]] for i in range(len1) for j in range(len2)]

In [5]:
def combinations(params):
    
    list_ = params
#     for value in params.values():
#         list_.append(value)
#     print(list_)
    tmp = reduce(lambda x, y: list_concat(x, y), list_)
    
    res = []
    for elem in tmp:
        param_list = flatten([elem])
#         print(param_list)
#         param_dict = list(zip(params.keys(), param_list))
        res.append(param_list)
        
    return res

In [59]:
class node():
    
    def __init__(self, level :int = -1, letter = None, N = 10):
        
        self.children = dict('')
        self.word = ''
        self.level = level
        self.letter = letter
        self.available = set('ьнс')
        self.N = N
    
    
    def language(self, word):
        
        eng = "qwertyuiop[]asdfghjkl;'zxcvbnm,"
        rus = "йцукенгшщзхъфывапролджэячсмитьб"
        rus_transfer = dict(zip(rus, eng))
        eng_transfer = dict(zip(eng, rus))
        
        keyboard, keyboard_t = (eng, eng_transfer) if word[0] in eng else (rus, rus_transfer)

        for i in range(len(word)):
            if word[i] not in keyboard:
                return False, word

        for i in range(len(word)):
            word = word[:i] + keyboard_t[word[i]] + word[i+1:]
            
        return True, word
                
        
    def add(self, word, position=0):
        
        if len(word) == position:
            self.word = word
            
        else:   
            if word[position] not in self.children.keys():
                self.children[word[position]] = node(self.level+1, word[position], self.N)
                
            self.children[word[position]].add(word, position + 1)
    
    
    def recount_probabilities(self, word, p, prev_letter):
        
        if self.level != -1:

            if self.letter == word[self.level]:
                new_p = p
            elif word[self.level] not in changes_dict or self.letter not in changes_dict[word[self.level]]:
                new_p = 0
            elif self.level == 0:
                new_p = p * sum(changes_dict[word[self.level]][self.letter].values())
            elif prev_letter not in changes_dict[word[self.level]][self.letter].keys():
                new_p = 0
            else:
                new_p = p * changes_dict[word[self.level]][self.letter][prev_letter]
        else:
            new_p = p
            
        return new_p
    
    
    def search_after_adding_letter(self, word, p, letter, prev_letter, letter_deleted):

        result = []
        condition = letter in inserts_dict.keys() and prev_letter in inserts_dict[letter].keys()

        if self.level > 0 and condition and not letter_deleted:

            word_after_insertion = word[:self.level+1] + letter + word[self.level+1:]
            insertion_p = p * inserts_dict[letter][prev_letter]/3 #3
            result += self.children[letter].search(word_after_insertion, insertion_p, self.letter, True)

        return result
    
    
    def search(self, word, p=1, prev_letter='', letter_added=False, letter_deleted=False, split=False):

        result = []
        p = self.recount_probabilities(word, p, prev_letter)

        if self.level == -1:
            for i in './+-=:':
                if i in word:
                    return []

        if p <= 1.0e-15:
            return []

        if self.level+1 == len(word):

            for node_letter in self.children:
                result += self.search_after_adding_letter(word, p, node_letter, self.letter, letter_deleted)

            result += [(self.word, p)] if self.word != '' else []

            return result

        else:
            for node_letter in self.children:
                result += self.search_after_adding_letter(word, p, node_letter, prev_letter, letter_deleted)
                result += self.children[node_letter].search(word, p, self.letter)

            if not letter_added and not letter_deleted:

                condition = self.letter in deletes_dict and prev_letter in deletes_dict[self.letter]

                if self.level >= 0 and condition:
                    d_p = p * deletes_dict[self.letter][prev_letter]/3
                    word_after_delete = word[:self.level] + word[self.level+1:]
                    result += self.search(word_after_delete, d_p, prev_letter, True, True)

        if self.level == -1:

            changed = self.language(word)

            if changed[0]:
                changed_word = changed[1]

                for node_letter in self.children:
                    result += self.children[node_letter].search(changed_word, p*med)

#             if len(result) == 0 and split == False:

#                 for split in range(1, len(word)):

#                     c = 1#0.04**abs(split-len(word)/2)

#                     r1 = self.search(word[:split], med*c/5, self.letter, split=True)
#                     r2 = self.search(word[split:], med*c/5, self.letter, split=True)

#                     r1 = r1 if r1 != [] else [(word[:split], 0)]
#                     r2 = r2 if r2 != [] else [(word[split:], 0)]

#                     p = r1[0][1] * r2[0][1]
#                     result += [(r1[0][0] + ' ' + r2[0][0], p)] if  p != 0 else []

        result.sort(key=lambda x: x[1])
        result = result[::-1]

        return result[:self.N]

In [7]:
class bore_search_tree():
    
    def __init__(self):
        
        self.main_node = node()
        
        
    def fit(self, words, freqs_sum, bigrams, bigrams_freqs_sum):
        
        self.words = words
        self.freqs_sum = freqs_sum
        self.bigrams_freqs_sum = bigrams_freqs_sum
        self.bigrams = bigrams

        for word in words.keys():
            self.main_node.add(word)
    
    
    def recount_probabilities(self, words, p, alpha=0.6):
        
        freq = 0

        for word_ in words.split():
            if word_ not in self.words:
                freq += 1 / self.freqs_sum
            else:
                freq += self.words[word_] / self.freqs_sum

        new_p = alpha*np.log(freq) + np.log(p)
        
        return words, new_p
        
    
    def search(self, word, top=10, alpha=0.6):
        
        res = self.main_node.search(word)[:top]

        for i in range(len(res)):
            res[i] = self.recount_probabilities(res[i][0], res[i][1], alpha)
        
        res.sort(key = lambda x: x[1])
        if res == []:
            res.append((word, 1))

        return res[::-1]
    
    
    def multisearch(self, words_list, top=10, alpha=0.6):
        
        res = []
        
        for word in words_list:
            res.append(self.search(word, top, alpha))
        
        return res
    
    
    def bigrams_with_probabilities(self, samples):
        
        res = []

        for sample in samples:
            if len(sample) > 1:
                for i in range(len(sample)-1, 0, -1):

                    bigram = sample[i-1][0] + ' ' + sample[i][0]

                    if bigram in self.bigrams:
                        freq = self.bigrams[bigram]
                    else:
                        freq = np.log(1/self.bigrams_freqs_sum)

                    sample.insert(i, (bigram, freq))

            res.append(sample)
        
        return res
    
    
    def answer_words_concatination(self, samples):
    
        final_res = []
    
        for sample in samples:

            fix = str()
            p = 0

            for i in range(0, len(sample), 2):
                fix += sample[i][0] + ' '

            for i in range(len(sample)):
                p += sample[i][1]

            final_res.append((fix[:-1], p))
        
        return final_res
    
    
    def classifier(self, answer):
        
        samples = combinations(answer)
        
        samples_with_bigrams = self.bigrams_with_probabilities(samples)
        
        generated_queries = self.answer_words_concatination(samples_with_bigrams)
        generated_queries.sort(key = lambda x: x[1])
        generated_queries = generated_queries[::-1]
    
        return generated_queries[:10]
    
    
    def correct_query(self, query, alpha=1.5):
        
        query = query.lower()
        words = query.split()
        
        answer = self.multisearch(words, alpha=alpha)
        
        words_num = len(words)
        num = 1 if words_num > 15 else 2 if words_num > 9 else 10 - words_num

        for i in range(len(answer)):
            answer[i] = answer[i][:num]
#         print(answer)
        bests = self.classifier(answer)
        
        return bests[0][0]

In [8]:
class Language_model():
    
    def __init__(self, queries):
        
        self.make_vectorizer(queries)
        self.bigrams_freqs(queries)
        
        
    def make_vectorizer(self, queries):
        
        vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
        X = vectorizer.fit_transform(queries)
        
        words = vectorizer.get_feature_names()
        freqs = np.array(X.sum(axis=0))[0]
        
        self.freqs_sum = freqs.sum()
    
        for i in range(len(words)):
            words[i] = words[i][:20]
          
        self.words = dict(zip(words[50:474500], freqs[50:474500]))
        
        
    def bigrams_freqs(self, queries):
    
        vectorizer = CountVectorizer(ngram_range=(2, 2))
        X = vectorizer.fit_transform(queries) 

        names = vectorizer.get_feature_names()
        freqs = np.array(X.sum(axis=0))[0]

        bad = np.where(freqs==1)[0][::-1]
        for idx in bad:
            names.pop(idx)

        self.bigrams_freqs_sum = freqs.sum()    
        freqs = freqs[np.where(freqs>1)[0]]/self.bigrams_freqs_sum
    
    #     res = []
    #     for i in range(freqs.shape[0]):
    #         res.append((names[i], freqs[i]/freqs_sum))
        self.bigrams = dict(zip(names, freqs))
    
    
    def get_bigrams(self):
        
        return self.bigrams, self.bigrams_freqs_sum
    
    
    def get_words(self):
        
        return self.words, self.freqs_sum

In [9]:
class Mistakes_model():
    
    def __init__(self, mistakes_with_correctings):
        
        self.mistakes_with_correctings = mistakes_with_correctings
        
        self.main_changes_dict = dict()
        self.main_inserts_dict = dict()
        self.main_deletes_dict = dict()
        
        self.dicts_making()
        
        self.main_changes_dict, self.changes_med, self.changes_mean = self.process_dict(self.main_changes_dict, 70)
        self.main_inserts_dict, self.inserts_med, self.inserts_mean = self.process_dict(self.main_inserts_dict,50)
        self.main_deletes_dict, self.deletes_med, self.deletes_mean = self.process_dict(self.main_deletes_dict,100)
        
        self.final_changes_dict_maker()
        self.final_inserts_dict_maker()
        self.final_deletes_dict_maker()

    
    def language_check(self, word1, word2):
    
        letters = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяabcdefghijklmnopqrstuvwxyz0123456789'

        for i in word1 + word2:
            if i not in letters:
                return False

        return True
    
        
    def levenstein_dist(self, word1, word2):
        main_matrix = np.zeros((len(word1)+1, len(word2)+1))

        for i in range(1, main_matrix.shape[0]):
            main_matrix[i, 0] = i

        for i in range(1, main_matrix.shape[1]):
            main_matrix[0, i] = i    

        for i in range(1, main_matrix.shape[0]):
            for j in range(1, main_matrix.shape[1]):

                minimum = min(main_matrix[i, j-1], main_matrix[i-1, j], main_matrix[i-1, j-1])

                if word1[i-1] == word2[j-1]:
                    main_matrix[i, j] = minimum
                else:
                    main_matrix[i, j] = minimum + 1

        return main_matrix        
        

    def stats(self, m):

        if m.shape[0] == 1 and m.shape[1] == 1:
            return []

        curr = m[-1, -1]
        res = []
        info = [0, 0, 0]

        if m.shape[0] > 1 and m[-2, -1] <= curr:
            res += [[m.shape[0]-2, m.shape[1]-1, -1]]
            info[0] = 1

        if m.shape[1] > 1 and m[-1, -2] <= curr:
            res += [[m.shape[0]-1, m.shape[1]-2,  1]]
            info[1] = 1

        if m.shape[0] > 1 and m.shape[1] > 1 and m[-2, -2] <= curr:
            res += [[m.shape[0]-2, m.shape[1]-2, 0]] 
            info[2] = 1

        if info[2] == 1:
            res += self.stats(m[:-1, :-1])
        else:
            if info[0] == 1:
                res += self.stats(m[:-1, :])
            if info[1] == 1:
                res += self.stats(m[:, :-1])

        return res     
    
    
    def letter_bigramms(self, word1, word2):
    
        if not self.language_check(word1, word2):
            return [], [], []
        
        m = self.levenstein_dist(word1, word2)
        for i in range(min(m.shape[0], m.shape[1])):
            if m[i][i] != 0:
                i -= 1
                break

        m = m[i:, i:]

        statistics = self.stats(m)
        for k in range(len(statistics)):
            statistics[k] = (statistics[k][0]+i, statistics[k][1]+i, statistics[k][2])

        statistics = list(set(statistics))  
        
        change_letter = []
        insert_letter = []
        delete_letter = []

        for info in statistics:

            if info[-1] == 0:
                change_letter.append((word2[info[1]-1] + word1[info[0]], word2[info[1]-1] + word2[info[1]]))
            if info[-1] == 1:
                insert_letter.append((word2[info[1]-1] + '*', word2[info[1]-1] + word2[info[1]]))
            if info[-1] == -1:
                delete_letter.append((word2[info[1]-1] + word1[info[0]], word2[info[1]-1] + '*'))

        change_num = 0

        while change_num < len(change_letter):
    #         print(res[change_num][0])
            if change_letter[change_num][0] == change_letter[change_num][1]:
                change_letter.pop(change_num)
            else:
                change_num += 1

        return change_letter, insert_letter, delete_letter
    
    
    def query_changes(self, query_pair):
    
        change_letter_list = []
        insert_letter_list = []
        delete_letter_list = []

        left = query_pair[0].split()
        right = query_pair[1].split()

        if len(left) == len(right):
            for word1, word2 in zip(left, right):
                if word1 != word2:

                    change_letter, insert_letter, delete_letter = self.letter_bigramms(word1, word2)

                    change_letter_list += change_letter
                    insert_letter_list += insert_letter
                    delete_letter_list += delete_letter

        return change_letter_list, insert_letter_list, delete_letter_list
    
    
    def dicts_making(self):
        
        for query_pair in self.mistakes_with_correctings:

            changes, inserts, deletes = self.query_changes(query_pair)

            for change in changes:
    #             if change[1][0] == 'и' and change[1][1] == 'ь':
    #                 print(query_pair)
                if change not in self.main_changes_dict.keys():
                    self.main_changes_dict[change] = 1
                else:
                    self.main_changes_dict[change] += 1

            for insert in inserts:
                if insert not in self.main_inserts_dict.keys():
                    self.main_inserts_dict[insert] = 1
                else:
                    self.main_inserts_dict[insert] += 1

            for delete in deletes:
                if delete not in self.main_deletes_dict.keys():
                    self.main_deletes_dict[delete] = 1
                else:
                    self.main_deletes_dict[delete] += 1

                    
    def process_dict(self, main_dict, threshold=100):
    
        new_dict = dict()

        for key in main_dict:
            if main_dict[key] >= threshold:
                new_dict[key] = main_dict[key]

        main_dict = new_dict
        print('current main_dict len = ', len(main_dict))

        dict_sum = sum(main_dict.values())

        for key in main_dict:
            main_dict[key] = main_dict[key]/dict_sum

        med = np.median(np.array([main_dict[k] for k in main_dict]))
        mean = np.mean(np.array([main_dict[k] for k in main_dict]))

        return main_dict, med, mean
    

    def final_changes_dict_maker(self):
    
        changes_dict = dict()

        for key in self.main_changes_dict:
            if key[0][1] not in changes_dict.keys():

                changes_dict[key[0][1]] = dict()

                if key[1][1] not in changes_dict[key[0][1]].keys():
                    changes_dict[key[0][1]][key[1][1]] = dict()
                    changes_dict[key[0][1]][key[1][1]][key[0][0]] = self.main_changes_dict[key]

                else:
                    changes_dict[key[0][1]][key[1][1]][key[0][0]] = self.main_changes_dict[key]

            else:

                if key[1][1] not in changes_dict[key[0][1]].keys():
                    changes_dict[key[0][1]][key[1][1]] = dict()
                    changes_dict[key[0][1]][key[1][1]][key[0][0]] = self.main_changes_dict[key]

                else:
                    changes_dict[key[0][1]][key[1][1]][key[0][0]] = self.main_changes_dict[key]

        self.main_changes_dict = changes_dict
    
    
    def final_inserts_dict_maker(self):
    
        inserts_dict = dict()

        for key in self.main_inserts_dict:

            if key[1][1] not in inserts_dict.keys():

                inserts_dict[key[1][1]] = dict()
                inserts_dict[key[1][1]][key[0][0]] = self.main_inserts_dict[key]

            else:

                inserts_dict[key[1][1]][key[0][0]] = self.main_inserts_dict[key]

        self.main_inserts_dict = inserts_dict
    
    
    def final_deletes_dict_maker(self):
    
        deletes_dict = dict()

        for key in self.main_deletes_dict:

            if key[0][1] not in deletes_dict.keys():

                deletes_dict[key[0][1]] = dict()
                deletes_dict[key[0][1]][key[0][0]] = self.main_deletes_dict[key]

            else:

                deletes_dict[key[0][1]][key[0][0]] = self.main_deletes_dict[key]

        self.main_deletes_dict = deletes_dict
    
    
    def get_dicts(self):
        
        return self.main_changes_dict, self.main_inserts_dict, self.main_deletes_dict
    
    
    def get_stats(self, stats='changes'):
        
        if stats == 'changes':
            return self.changes_med, self.changes_mean
        
        if stats == 'inserts':
            return self.inserts_med, self.inserts_mean
        
        if stats == 'deletes':
            return self.deletes_med, self.deletes_mean
        
        print('bad type')
        return 0, 0

In [10]:
queries, correct = queries_list()

In [11]:
%%time
m_model = Mistakes_model(correct)

current main_dict len =  197
current main_dict len =  551
current main_dict len =  234
CPU times: user 12.8 s, sys: 101 ms, total: 12.9 s
Wall time: 12.8 s


In [12]:
%%time
l_model = Language_model(queries)

CPU times: user 1min 43s, sys: 1.09 s, total: 1min 45s
Wall time: 1min 45s


In [13]:
bigrams, bigrams_freqs_sum = l_model.get_bigrams()
words, freqs_sum = l_model.get_words()

In [14]:
med, mean = m_model.get_stats('changes')

In [15]:
changes_dict, inserts_dict, deletes_dict = m_model.get_dicts()

In [60]:
b = bore_search_tree()

In [61]:
%%time
b.fit(words, freqs_sum, bigrams, bigrams_freqs_sum)

CPU times: user 14.9 s, sys: 863 ms, total: 15.8 s
Wall time: 16.3 s


In [33]:
queries[302]

'вытащила член взвешивая его в ладони плотоядно такой молодой и уже такой богатый порно рассказ читать\n'

In [91]:
# q = 'онреей молохов gecnm гаварят'
# q = 'gjujlf'
q = 'gfulf d Масве ctujlz'
# q = 'ctujlz'
q = 'gjhyj ne,t'
q = 'порна видо сматрет безплатна'
# q = 'игрет в тенес'
# q = 'gfuljf'
# q = 'google.com'
# q = 'смотретьогурец'
# q = 'мсква слзам нее верт'
# q = 'где можно найти книгу советского горнолыжника тальянова'
# q = 'беременною порно'
q = 'что входитв стоимиость авиабилетов'
q = 'как pfvtybnm gjgkfdjr d eybnfpt c jrjdjq gjldjlrjq'

In [58]:
len(q.split())

4

In [92]:
%%time
b.correct_query(q, 2)

CPU times: user 168 ms, sys: 35.8 ms, total: 204 ms
Wall time: 259 ms


'как заменить поплавок в унитазе с акуловой подводкой'

In [48]:
med

0.0032662884648444216

In [58]:
deletes_dict['п']

{'п': 0.004548084829903658,
 'л': 0.0010456534318751714,
 'р': 0.0019593311878826026,
 'а': 0.00292376881922378,
 'б': 0.002223282539618083}

In [42]:
inserts_dict['о']

{'р': 0.011354580361318027,
 'н': 0.008344694499878156,
 'к': 0.012625713777637272,
 'м': 0.00787048928757253,
 'е': 0.001514822205976303,
 'г': 0.003536780541779455,
 'с': 0.0038068140654534915,
 'д': 0.002792541805799793,
 'а': 0.0005927565153820316,
 'т': 0.006270046696041045,
 'п': 0.005868289502282112,
 'о': 0.0010142722596536984,
 'л': 0.0054533599415146905,
 'и': 0.0016004425915314853,
 'в': 0.006349080898091983,
 'ш': 0.0005005499463226045,
 'ф': 0.001508236022472058,
 'з': 0.0016992353440951572,
 'ь': 0.0008101005710221098,
 'б': 0.0026937490532361213,
 'й': 0.0009286518740985162,
 'х': 0.001448960370933855,
 'ч': 0.0010537893606791672,
 'я': 0.0003754124597419533,
 'ж': 0.00032930917521223975}

In [61]:
%%time
with open('bore_tree.pickle', 'wb') as f:
    pickle.dump(b, f)
    


CPU times: user 11.4 s, sys: 481 ms, total: 11.9 s
Wall time: 11.8 s


In [52]:
%%time
with open('bore_tree.pickle', 'rb') as f:
    data_new = pickle.load(f)

CPU times: user 17.3 s, sys: 488 ms, total: 17.8 s
Wall time: 17.8 s


In [28]:
data_new.correct_query(q)

'андрей малахов пусть говорят'

In [32]:
%%time
with open('l_model.pickle', 'wb') as f:
    pickle.dump(l_model, f)

CPU times: user 3.03 s, sys: 148 ms, total: 3.17 s
Wall time: 3.16 s


In [33]:
%%time
with open('bigrams.pickle', 'rb') as f:
    data_new = pickle.load(f)

CPU times: user 2.78 s, sys: 160 ms, total: 2.94 s
Wall time: 2.93 s


In [39]:
import gzip


In [42]:
gzip.compress(b, compresslevel=9, mtime=None)

TypeError: compress() got an unexpected keyword argument 'mtime'

In [43]:
import json

In [57]:
run  = True

while run:

    try:
        q = input()
        print(q)
    except EOFError:
        run = False

dfbd
dfbd
dsfb
dsfb
dfb
dfb
dfb
dfb
dfb
dfb
\0
\0
0
0


KeyboardInterrupt: 